In [31]:
# -*- coding: utf-8 -*-
"""
🎯 (2024+2025) 세분화된 분류(대/중/세부) 기반 축제 분류 · 근거 · 동일세부유형 리스트
- 로드: ./csv/{2024.csv, 2025.csv} (+ /mnt/data 경로도 시도)
- 아직 시작 안 한 축제(시작일 > 오늘) 제외
- 입력 축제(또는 매칭명) 제외
- 분류 우선순위: dataset ▶ rule ▶ llm(선택)
- 출력:
    1) 탐지된 [대/중/세부] 유형 (+근거)
    2) 동일 (세부→중→대) 유형 축제 수
    3) 동일 유형 축제 목록(이름만)
"""

import os, re, json
from pathlib import Path
from typing import Optional, Tuple, List, Dict
from difflib import get_close_matches

import pandas as pd
from dotenv import load_dotenv

# ---------------- Env / OpenAI (선택) ----------------
load_dotenv()
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

_client_mode = None
try:
    from openai import OpenAI  # >=1.x
    _client = OpenAI()
    _client_mode = "new"
except Exception:
    try:
        import openai               # <=0.x
        openai.api_key = os.getenv("OPENAI_API_KEY", "")
        _client = openai
        _client_mode = "legacy"
    except Exception:
        _client = None
        _client_mode = None

# ---------------- 파일 경로 ----------------
CANDIDATE_FILES = [
    "./csv/2024.csv",
    "./csv/2025.csv",
    "/mnt/data/2024.csv",
    "/mnt/data/2025.csv",
]

# ---------------- 분류 체계 ----------------
ALLOWED_MAJOR = ["문화예술", "자연생태", "전통역사", "지역특산물"]

# 세부 규칙(키워드) : 나비/반딧불 ⇒ '곤충'로 흡수
FINE_RULES: Dict[str, Dict[str, List[str]]] = {
    "자연생태": {
        "꽃": ["꽃","벚꽃","장미","튤립","코스모스","유채","단풍","억새"],
        "곤충": ["곤충","나비","반딧불","애벌레","잠자리","사슴벌레"],
        "조류·철새": ["철새","두루미","기러기","물새","갈매기"],
        "천문·별": ["천문","별","은하","유성","천체","야경"],
        "숲·생태원": ["생태","습지","숲","수목원","자연","생태원"],
        "산·계곡·물": ["산","계곡","호수","강","바다","해변","섬"],
    },
    "지역특산물": {
        "과일": ["사과","포도","딸기","수박","밤","복숭아","자두","귤","매실","블루베리"],
        "곡물·채소": ["감자","고구마","고추","옥수수","쌀","보리","콩","배추","무","양파"],
        "수산·해산물": ["굴","장어","수산","해산물","전복","낙지","문어","멍게","새우","게","미역","김"],
        "축산": ["한우","한돈","우유","치즈","양고기"],
        "주류": ["와인","맥주","막걸리","전통주","소주","술"],
        "디저트·카페": ["커피","빵","베이커리","디저트","케이크","초코","쿠키"],
        "발효·김치": ["김치","젓갈","장","된장","고추장","간장","장아찌"],
    },
    "문화예술": {
        "음악·콘서트": ["dj","edm","힙합","랩","kpop","케이팝","뮤직","음악","콘서트","페스티벌","재즈","클래식","버스킹","합창","연주","오케스트라"],
        "무용·댄스": ["댄스","무용","비보이","댄싱"],
        "연극·뮤지컬·영화": ["연극","뮤지컬","영화","영화제","시네마"],
        "전시·미술·사진": ["전시","미술","아트","사진","비엔날레","트리엔날레"],
        "국악·전통공연": ["국악","판소리","사물놀이","풍물","탈춤"],
        "시즌·겨울": ["산타","크리스마스","연말","송년"],
    },
    "전통역사": {
        "전통공예": ["도자기","도예","옹기","한지","서예","목공","금속공예","염색","옻칠"],
        "유적·건축": ["한옥","고택","서원","향교","읍성","성곽","고분","왕릉","사찰","궁"],
        "민속·향토": ["전통","전통축제","민속","향토","세시","의례","단오","정월대보름","풍어제","당산제"],
        "무형문화재": ["무형문화재","국가무형문화재","전승"],
    },
}

# 역매핑(세부 → 대)
FINE_TO_MAJOR = {fine: major for major, bucket in FINE_RULES.items() for fine in bucket.keys()}

# ---------------- 유틸 ----------------
def normalize_text(s: str) -> str:
    if s is None: return ""
    s = str(s).strip().replace("\xa0", " ")
    return re.sub(r"\s+", "", s)

def parse_date_str(s: str) -> Optional[pd.Timestamp]:
    return pd.to_datetime(str(s), errors="coerce", format="%Y-%m-%d")

def today_floor_ts() -> pd.Timestamp:
    return pd.Timestamp.today().normalize()

# 동의어를 표준 컬럼명으로 정규화
COL_SYNONYMS = {
    "연번": ["연번", "번호", "id", "ID"],
    "광역": ["광역자치단체명","광역","시도","광역시도","광역명"],
    "기초": ["기초자치단체명","기초","시군구","자치구","기초명"],
    "축제명": ["축제명","행사명","이벤트명","명칭","타이틀"],
    "시작일": ["시작일","start","시작","start_date","시작일자","개막일"],
    "종료일": ["종료일","end","종료","end_date","종료일자","폐막일"],
    "대분류": ["대분류","분류대","유형대","축제유형(대)","축제유형_대","분류_대","대분류명"],
    "중분류": ["중분류","분류중","유형중","축제유형(중)","축제유형_중","분류_중","중분류명"],
    "세부분류": ["세부분류","세부유형","소분류","세부분류","분류소","유형세부","축제유형(세부)","축제유형_세부","세부분류명"],
    # 구버전 단일열
    "구_축제유형": ["축제유형","축제 유형"],
}

def _pick_first_exist(colnames: List[str], df_cols: List[str]) -> Optional[str]:
    s = set(df_cols)
    for c in colnames:
        if c in s: return c
    return None

def standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    mapping = {}
    cols = [str(c).strip() for c in df.columns]
    for std, synonyms in COL_SYNONYMS.items():
        found = _pick_first_exist(synonyms, cols)
        if found:
            mapping[found] = std
    out = df.rename(columns=mapping).copy()
    # 표준 컬럼이 없으면 빈 컬럼 추가
    for need in ["연번","광역","기초","축제명","시작일","종료일","대분류","중분류","세부분류","구_축제유형"]:
        if need not in out.columns:
            out[need] = ""
    # 구버전 단일열(축제유형) → 대분류/세부분류 추정
    if out["구_축제유형"].astype(str).str.strip().ne("").mean() > 0:
        # 값이 대부분 ALLOWED_MAJOR면 대분류로, 아니면 세부분류로 간주
        vals = out["구_축제유형"].astype(str).str.strip()
        ratio_major = vals.isin(ALLOWED_MAJOR).mean()
        if ratio_major >= 0.6:
            out.loc[out["대분류"].eq(""), "대분류"] = vals
        else:
            out.loc[out["세부분류"].eq(""), "세부분류"] = vals
    # 세부분류 → 대분류 자동 채움(룰 테이블 기준)
    miss_major = out["대분류"].astype(str).str.strip().eq("")
    guess = out.loc[miss_major, "세부분류"].map(lambda x: FINE_TO_MAJOR.get(str(x).strip(), ""))
    out.loc[miss_major & guess.astype(bool), "대분류"] = guess
    return out

def ensure_columns(df: pd.DataFrame) -> pd.DataFrame:
    needed = ["연번","광역","기초","축제명","시작일","종료일","대분류","중분류","세부분류"]
    for c in needed:
        if c not in df.columns:
            raise ValueError(f"필수 컬럼 누락: {c}")
    return df

def read_many_csv(paths: List[str]) -> pd.DataFrame:
    frames = []
    for p in paths:
        if Path(p).exists():
            try:
                df = pd.read_csv(p, dtype=str).fillna("")
            except UnicodeDecodeError:
                df = pd.read_csv(p, dtype=str, encoding="utf-8-sig").fillna("")
            df["_출처파일"] = Path(p).name
            df = standardize_columns(df)
            df = ensure_columns(df)
            # 혹시 남아있는 '주민화합'은 제거
            mask = ~(
                df["대분류"].astype(str).str.contains("주민화합", na=False) |
                df["중분류"].astype(str).str.contains("주민화합", na=False) |
                df["세부분류"].astype(str).str.contains("주민화합", na=False)
            )
            df = df[mask]
            frames.append(df)
    if not frames:
        raise FileNotFoundError("입력 CSV를 하나도 찾지 못했습니다.")
    out = pd.concat(frames, ignore_index=True)
    out = out.drop_duplicates(
        subset=["광역","기초","축제명","대분류","중분류","세부분류","시작일","종료일"],
        keep="first"
    ).reset_index(drop=True)
    return out

# ---------------- 분류(데이터셋/규칙/LLM) ----------------
def _extract_json(text: str) -> Optional[dict]:
    if not text: return None
    m = re.search(r"\{.*\}", text, flags=re.S)
    if not m: return None
    try:
        return json.loads(m.group(0))
    except Exception:
        return None

def dataset_detect(df: pd.DataFrame, name: str) -> Tuple[str,str,str, dict]:
    """데이터셋에서 [대,중,세부]를 찾아서 반환. 없으면 빈 문자열."""
    exact = df[df["축제명"].astype(str).str.strip() == name.strip()]
    if exact.empty:
        # 부분 포함
        nname = normalize_text(name)
        cand = df[df["축제명"].astype(str).apply(lambda x: normalize_text(x).find(nname) >= 0 or nname.find(normalize_text(x)) >= 0)]
        if not cand.empty:
            exact = cand
        else:
            # 유사도 매칭
            hits = get_close_matches(name, df["축제명"].astype(str).tolist(), n=1, cutoff=0.75)
            if hits:
                exact = df[df["축제명"] == hits[0]]

    if exact.empty:
        return "","","", {}

    row = exact.iloc[0]
    major = str(row.get("대분류","")).strip()
    mid   = str(row.get("중분류","")).strip()
    fine  = str(row.get("세부분류","")).strip()

    ev = {
        "source": "dataset",
        "file": str(row.get("_출처파일","")),
        "연번": str(row.get("연번","")),
        "matched_name": str(row.get("축제명","")),
    }
    return major, mid, fine, ev

def collect_rule_hits(name: str) -> Dict[str, Dict[str, List[str]]]:
    nm = (name or "").lower()
    nm_plain = re.sub(r"\s+", "", nm)
    hits: Dict[str, Dict[str, List[str]]] = {}
    for major, bucket in FINE_RULES.items():
        for fine, kws in bucket.items():
            for kw in kws:
                k = kw.lower()
                if (k in nm) or (k in nm_plain):
                    hits.setdefault(major, {}).setdefault(fine, [])
                    if kw not in hits[major][fine]:
                        hits[major][fine].append(kw)
    return hits

def rule_detect(name: str) -> Tuple[str,str,dict]:
    """규칙으로 [대,세부] 추정"""
    hits = collect_rule_hits(name)
    if not hits: return "","", {}
    # 점수화: fine hit 수가 가장 큰 조합
    best = None
    best_score = -1
    for major, fine_map in hits.items():
        for fine, kws in fine_map.items():
            score = len(kws)
            if score > best_score:
                best = (major, fine, kws)
                best_score = score
    if best is None: return "","", {}
    major, fine, kws = best
    ev = {"source":"rule","major":major,"fine":fine,"hits":", ".join(kws)}
    return major, fine, ev

def llm_detect(name: str, allowed_fine: List[str], allowed_major: List[str]) -> Tuple[str,str,dict]:
    if _client_mode is None or _client is None:
        return "","", {}
    allowed_fine = sorted(list({f for f in allowed_fine if f and f not in ["-", ""]}))[:60]  # 안전상 60개 제한
    majors_str = "|".join(allowed_major)
    fines_str  = "|".join(allowed_fine) if allowed_fine else ""
    prompt = f"""
한국 축제명: "{name}"

아래 JSON만 출력:
{{
  "major": "{majors_str}",
  "fine": "{fines_str if fines_str else '<없으면 빈 문자열>'}",
  "hint_keywords": ["<이름에서 포착한 힌트 단어 최대 3개>"]
}}
규칙:
- major는 반드시 위 4개 중 하나(문화예술/자연생태/전통역사/지역특산물)
- fine은 제공된 목록에서 고르되, 적합한게 없으면 ""(빈 문자열)
- 설명문 금지. JSON만.
"""
    try:
        if _client_mode == "new":
            resp = _client.chat.completions.create(
                model=OPENAI_MODEL,
                messages=[
                    {"role":"system","content":"Return ONLY compact JSON."},
                    {"role":"user","content":prompt},
                ],
                temperature=0, max_tokens=80,
            )
            content = resp.choices[0].message.content.strip()
        else:
            resp = _client.ChatCompletion.create(
                model=OPENAI_MODEL,
                messages=[
                    {"role":"system","content":"Return ONLY compact JSON."},
                    {"role":"user","content":prompt},
                ],
                temperature=0, max_tokens=80,
            )
            content = resp["choices"][0]["message"]["content"].strip()
        data = _extract_json(content) or {}
        major = str(data.get("major","")).strip()
        fine  = str(data.get("fine","")).strip()
        hints = data.get("hint_keywords", [])
        if not isinstance(hints, list): hints = []
        ev = {"source":"llm","major":major,"fine":fine,"hints":", ".join(map(str,hints))}
        return major, fine, ev
    except Exception:
        return "","", {}

# ---------------- 파이프라인 ----------------
def detect_labels(df: pd.DataFrame, name: str) -> Tuple[str,str,str,dict]:
    """[대,중,세부, Evidence]"""
    major, mid, fine, ev = dataset_detect(df, name)
    if major or mid or fine:
        # 세부가 비었는데 데이터에 중분류만 있으면 fine<-중으로 승격
        if fine == "" and mid != "":
            fine = mid
        return major, mid, fine, {"source":"dataset", **ev}

    # 규칙
    r_major, r_fine, r_ev = rule_detect(name)
    if r_major or r_fine:
        # mid는 아직 모르면 빈칸
        return r_major, "", r_fine, r_ev

    # LLM (선택)
    allowed_fine = sorted(df["세부분류"].astype(str).str.strip().unique().tolist())
    l_major, l_fine, l_ev = llm_detect(name, allowed_fine, ALLOWED_MAJOR)
    if l_major or l_fine:
        return l_major, "", l_fine, l_ev

    return "","","", {"source":"none"}

def filter_same_group(df: pd.DataFrame, name: str, major: str, mid: str, fine: str) -> pd.DataFrame:
    """동일 '세부→중→대' 그룹에서, 오늘 이전 시작 축제만, 자기 자신 제외"""
    base = df.copy()
    # 우선순위: 세부 > 중 > 대
    if fine:
        mask = base["세부분류"].astype(str).str.strip() == fine
    elif mid:
        mask = base["중분류"].astype(str).str.strip() == mid
    else:
        mask = base["대분류"].astype(str).str.strip() == major
    same = base[mask].copy()

    # 아직 시작 안 한 축제 제외
    today = today_floor_ts()
    same["_시작"] = pd.to_datetime(same["시작일"], errors="coerce")
    same = same[(~same["_시작"].isna()) & (same["_시작"] <= today)]

    # 자기 자신 제외(이름 정규화)
    inorm = normalize_text(name)
    same = same[ same["축제명"].astype(str).apply(lambda x: normalize_text(x) != inorm) ]

    same = same.sort_values(by=["광역","기초","축제명"]).drop(columns=["_시작"], errors="ignore")
    return same[["연번","광역","기초","축제명","대분류","중분류","세부분류","시작일","종료일","_출처파일"]]

def run(festival_name: str, print_limit: int = 300, dedupe: bool = True):
    # 데이터 로드
    df = read_many_csv(CANDIDATE_FILES)

    # 라벨 탐지
    major, mid, fine, ev = detect_labels(df, festival_name)

    # 출력 헤더
    print(f"입력 축제명: {festival_name}")
    if fine or mid or major:
        print(f"탐지된 유형: 대='{major or '-'}' | 중='{mid or '-'}' | 세부='{fine or '-'}'  | 출처: {ev.get('source','-')}")
    else:
        print("탐지된 유형: (탐지 실패)")
    # 근거
    src = ev.get("source","-")
    if src == "dataset":
        print(f"근거(dataset): 파일={ev.get('file','')}, 연번={ev.get('연번','')}, 매칭명={ev.get('matched_name','')}")
    elif src == "rule":
        print(f"근거(rule): major='{ev.get('major','')}', fine='{ev.get('fine','')}', 히트=[{ev.get('hits','')}]")
    elif src == "llm":
        print(f"근거(llm): major='{ev.get('major','')}', fine='{ev.get('fine','')}', 힌트=[{ev.get('hints','')}]")
    else:
        print("근거: -")

    # 동일 그룹 추출
    same = filter_same_group(df, festival_name, major, mid, fine)

    if same.empty:
        print("\n동일 유형 축제 수: 0")
        return df, same

    if dedupe:
        same = same.drop_duplicates(subset=["축제명"], keep="first")

    # 리스트 출력(이름만)
    names = same.sort_values(by=["시작일","축제명"])
    names_list = names["축제명"].astype(str).tolist()

    print(f"\n동일 유형 축제 수: {len(names_list)}")
    print("\n[동일 유형 축제 목록]")
    for i, nm in enumerate(names_list[:print_limit], start=1):
        print(f"{i:>3}. {nm}")
    if len(names_list) > print_limit:
        print(f"... ({len(names_list) - print_limit}개 더 있음)")

    return df, same

# ------------- Example -------------
if __name__ == "__main__":
    # 예시: 이름만 바꿔 테스트
    _df, _same = run("담양산타축제")


입력 축제명: 담양산타축제
탐지된 유형: 대='-' | 중='-' | 세부='문화예술-크리스마스·산타'  | 출처: dataset
근거(dataset): 파일=2025.csv, 연번=793, 매칭명=제7회 담양산타축제

동일 유형 축제 수: 2

[동일 유형 축제 목록]
  1. 제13회 유러피안 크리스마스 마켓
  2. 2024 유성온천 크리스마스축제
